# Matching SpenderQ output to Picca "delta_extraction" format

This notebook is a copy of the SpenderQ_to_Picca.ipynb. Changes include:
- creating a loop over ibatch for the entire notebook. This means the initial spenderq arrays are reset during each iteration.
- when saving the delta files they are saved as delta-ibatch instead of delta-ichunk, since with the new loop there is only one ichunk per ibatch (given the chunk size of 1024 remains unchanged). 

Future (potential) updates include changes to save the delta files under their healpix number instead of ibatch number, but it remains to be seen if this is a problem with picca yet. 

This notebook takes SpenderQ outputs, and reformats it to match the "delta files" format obtained by Picca via "delta_extraction"; making it compatible with picca to calculate the correlation functions. [2025 February 11]

In [1]:
import pickle
import numpy as np
import glob
import torch
import math

In [2]:
from astropy.io import fits
from astropy.table import Table

In [3]:
from spenderq import load_model
from spenderq import util as U
from spenderq import lyalpha as LyA

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
#import healpy as hp

## Add paths, define variables, load catalogue

In [6]:
#quasar catalogue of the production of interest (fits file)

## EDR
#quasar_catalogue = '/global/cfs/projectdirs/desi/users/sgontcho/lya/spender-lya/QSO_cat_EDR_n_M2_main_dark_healpix_BAL_n_DLA_cuts.fits'

## Y1 LONDON MOCKS
# quasar_catalogue = '/global/cfs/cdirs/desicollab/mocks/lya_forest/develop/london/qq_desi/v9.0_Y1/v9.0.9.9.9/desi-4.124-4-prod/zcat.fits'
quasar_catalogue = '/global/cfs/cdirs/desi/users/abault/spenderq/zcat_hpx.fits'

In [7]:
#NOTE THESE FUNCTIONS WILL ONLY RUN IF healpy was installed and imported above
#we created a separate catalog that added the healpix information, path is above

def calculate_healpix(catalog, nside):
    #calculates healpix values given a catalog and nside value
    hpx = hp.ang2pix(nside, np.pi/2-np.deg2rad(catalog['DEC']), np.deg2rad(catalog['RA']))
    catalog['HPXPIXEL'] = hpx

def add_healpix(catalog, nside = 8):
    #if healpix is not already in catalog, calculate and add a column
    #desi data catalogs use 'HPXPIXEL' as column name
    #default nside value in picca_cf is 16, this produced 1032 healpixs which I thought was too many, so I went with nside=8 (302)
    
    if 'HPXPIXEL' in catalog.columns:
        return catalog
    else:
        #add healpix to catalog
        catalog = calculate_healpix(catalog, nside)
        return catalog

In [41]:
#point to the SpenderQ output directory and give the files prefix (from the same production as the catalogue!)

path = '/global/cfs/projectdirs/desi/users/chahah/spender_qso'
#outpath = '/global/cfs/projectdirs/desi/users/sgontcho/lya/spender-lya/spenderq-to-deltas'
# outpath = '/global/cfs/projectdirs/desi/users/sgontcho/lya/spender-lya/iron_comparison/spenderq_prod_20241126_v0/Delta'
outpath = '/global/cfs/projectdirs/desi/users/abault/spenderq/deltas_lya/test_hpx'

## Y1 LONDON MOCKS
file_prefix = 'spenderq_london_v0/DESIlondon_highz.rebin.iter3'
spender_output_files = glob.glob(path+'/'+file_prefix+'_*.pkl')

## EDR
#file_prefix = 'DESI.edr.qso_highz'
#spender_output_files = glob.glob('/global/cfs/projectdirs/desi/users/chahah/spender_qso/DESIedr.qso_highz_*.pkl')

In [9]:
#LAMBDA range defined in picca
picca_lambda_min = 3600.
picca_lambda_max = 5772.

#FOREST range (default is LyA)
forest_lambda_min = 1040.
forest_lambda_max = 1205.

In [27]:
hdul = fits.open(quasar_catalogue)

In [28]:
catalogue_TID = hdul[1].data['TARGETID']
catalogue_Z = hdul[1].data['Z']
catalogue_RA = hdul[1].data['RA']
catalogue_DEC = hdul[1].data['DEC']
catalogue_HPX = hdul[1].data['HPXPIXEL']

##EDR
#catalogue_RA = hdul[1].data['TARGET_RA']
#catalogue_DEC = hdul[1].data['TARGET_DEC']

In [12]:
LAMBDA = np.arange(picca_lambda_min,picca_lambda_max+0.8,0.8)

In [15]:
#grab all values from spenderq and merge into one array
normalised_flux, pipeline_weight, z, tid, normalisation, zerr = [], [], [], [], [], []

sq_reconstructions = []

for ibatch in range(50):
    # print(ibatch)
#for ibatch in range(len(spender_output_files)): 
    
    #load batch
    with open(f'{path}/{file_prefix}_%i.pkl' % ibatch, 'rb') as f:
        _normalised_flux, _pipeline_weight, _z, _tid, _normalisation, _zerr = pickle.load(f)
    normalised_flux.append(np.array(_normalised_flux))
    pipeline_weight.append(np.array(_pipeline_weight))
    z.append(np.array(_z))
    tid.append(np.array(_tid))
    normalisation.append(np.array(_normalisation))
    zerr.append(np.array(_zerr))
    
    #load SpenderQ reconstructions
    _sq_reconstructions = np.load(f'{path}/{file_prefix}_%i.recons.npy' % ibatch)
    sq_reconstructions.append(np.array(_sq_reconstructions))

normalised_flux=np.concatenate(normalised_flux,axis=0)
pipeline_weight=np.concatenate(pipeline_weight,axis=0)
z=np.concatenate(z)
tid=np.concatenate(tid)
normalisation=np.concatenate(normalisation,axis=0)
zerr=np.concatenate(zerr,axis=0)
sq_reconstructions=np.concatenate(sq_reconstructions,axis=0)

In [18]:
#nb_of_quasars = 10
nb_of_quasars = len(z)

_tff = np.full((nb_of_quasars,7781),np.nan)
_weight = np.full((nb_of_quasars,7781),np.nan)
_sq_cont = np.full((nb_of_quasars,7781),np.nan)

wrecon = np.load(f'{path}/{file_prefix}.wave_recon.npy')
desi_wave = np.linspace(3600, 9824, 7781) #obs

_tff_bar_up = np.zeros(7781)
_tff_bar_down = np.zeros(7781)

#for iqso in range(10):
for iqso in range(nb_of_quasars):

    #create wavelength grids
    desi_wave_rest = desi_wave/(1+z[iqso]) #rest
    
    #rebin spender reconstruction
    edges = np.linspace(wrecon[0], wrecon[-1], 7782)
    spenderq_rebin = U.trapz_rebin(wrecon, sq_reconstructions[iqso], edges = edges)
    
    #keep only part of spec that is within the lya range
    mask_desi_rest = (desi_wave_rest >= forest_lambda_min) & (desi_wave_rest <= forest_lambda_max)

    _tff[iqso][mask_desi_rest] = normalised_flux[iqso][mask_desi_rest]/spenderq_rebin[mask_desi_rest]
    _weight[iqso][mask_desi_rest] = pipeline_weight[iqso][mask_desi_rest]
    _sq_cont[iqso][mask_desi_rest] = spenderq_rebin[mask_desi_rest]
    _tff_bar_up += normalised_flux[iqso]/spenderq_rebin * pipeline_weight[iqso]
    _tff_bar_down += pipeline_weight[iqso]


In [19]:
#get the weighted average transmitted flux fraction 
picca_range = (desi_wave <= LAMBDA[-1])
tff_bar = np.divide(_tff_bar_up[picca_range], _tff_bar_down[picca_range], out=np.zeros_like(_tff_bar_up[picca_range]), where=_tff_bar_down[picca_range]!=0)

In [20]:
#picca structure: nan filled arrays
delta = np.full((nb_of_quasars,sum(picca_range)),np.nan)
weight = np.full((nb_of_quasars,sum(picca_range)),np.nan)
sq_cont = np.full((nb_of_quasars,sum(picca_range)),np.nan)

meta_los_id, meta_ra, meta_dec, meta_z, meta_meansnr, meta_targetid, meta_night, meta_petal, meta_tile = [], [], [], [], [], [], [], [], []

#for iqso in range(5):
for iqso in range(nb_of_quasars):
    
    delta[iqso] = (_tff[iqso][picca_range]/tff_bar) - 1.
    weight[iqso] = _weight[iqso][picca_range]
    sq_cont[iqso] = _sq_cont[iqso][picca_range]
    
    los_id_idx = int(np.where(catalogue_TID == int(tid[iqso]))[0][0])
    
    meta_los_id.append(int(catalogue_TID[los_id_idx]))
    meta_ra.append(float(math.radians(catalogue_RA[los_id_idx])))
    meta_dec.append(float(math.radians(catalogue_DEC[los_id_idx])))
    meta_z.append(float(catalogue_Z[los_id_idx]))
    meta_meansnr.append(np.nan)
    meta_targetid.append(int(catalogue_TID[los_id_idx]))
    meta_night.append('')
    meta_petal.append('')
    meta_tile.append('')
    

In [22]:
# Create primary HDU 
primary_hdu = fits.PrimaryHDU(data=None)

# Create OBSERVED WAVELENGTH HDU
hdu_wave = fits.ImageHDU(LAMBDA, name=f'LAMBDA')
hdu_wave.header['HIERARCH WAVE_SOLUTION'] = 'lin'
hdu_wave.header['HIERARCH DELTA_LAMBDA'] = 0.8

In [48]:
#loop over healpix to get which qsos to save in which delta file:
unique_hpx = np.unique(catalogue_HPX)

for hpx in unique_hpx:
    hpx_mask = catalogue_HPX == hpx
    tids_hpx = catalogue_TID[hpx_mask]

    spender_mask = np.isin(tid, tids_hpx)
    
    if np.count_nonzero(spender_mask) != 0:
        c1 = fits.Column(name='LOS_ID', format='K', array=np.array(meta_los_id)[spender_mask])
        c2 = fits.Column(name='RA', format='D', array=np.array(meta_ra)[spender_mask])
        c3 = fits.Column(name='DEC', format='D', array=np.array(meta_dec)[spender_mask])
        c4 = fits.Column(name='Z', format='D', array=np.array(meta_z)[spender_mask])
        c5 = fits.Column(name='MEANSNR', format='D', array=np.array(meta_meansnr)[spender_mask])
        c6 = fits.Column(name='TARGETID', format='K', array=np.array(meta_targetid)[spender_mask])
        c7 = fits.Column(name='NIGHT', format='12A', array=np.array(meta_night)[spender_mask])
        c8 = fits.Column(name='PETAL', format='12A', array=np.array(meta_petal)[spender_mask])
        c9 = fits.Column(name='TILE', format='12A', array=np.array(meta_tile)[spender_mask])
        hdu_meta = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9], name='METADATA')
        hdu_meta.header['BLINDING'] = 'none'
    
        hdu_delta = fits.ImageHDU(delta[spender_mask], name=f'DELTA')
        hdu_weight = fits.ImageHDU(weight[spender_mask], name=f'WEIGHT')
        hdu_cont = fits.ImageHDU(sq_cont[spender_mask], name=f'CONT')
        hdu_tff_bar = fits.ImageHDU(tff_bar, name=f'FBAR')
    
        hdul1 = fits.HDUList([primary_hdu, hdu_wave, hdu_meta, hdu_delta, hdu_weight, hdu_cont, hdu_tff_bar])
    
        # Write the HDUList to a new FITS file
        hdul1.writeto(f'{outpath}/delta-{hpx}.fits', overwrite=True)

In [ ]:
# Create primary HDU 
primary_hdu = fits.PrimaryHDU(data=None)

# Create OBSERVED WAVELENGTH HDU
hdu_wave = fits.ImageHDU(LAMBDA, name=f'LAMBDA')
hdu_wave.header['HIERARCH WAVE_SOLUTION'] = 'lin'
hdu_wave.header['HIERARCH DELTA_LAMBDA'] = 0.8

# Set chunk size, and how to chunk
chunk_size = 1024

def chunks(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]

_delta_chunks = chunks(delta, chunk_size)
_weight_chunks = chunks(weight, chunk_size)
_cont_chunks = chunks(sq_cont, chunk_size)

nb_of_chunks = len(_delta_chunks)
print(nb_of_chunks)

for ichunk in range(nb_of_chunks):
    i=0
    c1 = fits.Column(name='LOS_ID', format='K', array=np.array(meta_los_id[i*chunk_size:(i+1)*chunk_size]))
    c2 = fits.Column(name='RA', format='D', array=np.array(meta_ra[i*chunk_size:(i+1)*chunk_size]))
    c3 = fits.Column(name='DEC', format='D', array=np.array(meta_dec[i*chunk_size:(i+1)*chunk_size]))
    c4 = fits.Column(name='Z', format='D', array=np.array(meta_z[i*chunk_size:(i+1)*chunk_size]))
    c5 = fits.Column(name='MEANSNR', format='D', array=meta_meansnr[i*chunk_size:(i+1)*chunk_size])
    c6 = fits.Column(name='TARGETID', format='K', array=np.array(meta_targetid[i*chunk_size:(i+1)*chunk_size]))
    c7 = fits.Column(name='NIGHT', format='12A', array=meta_night[i*chunk_size:(i+1)*chunk_size])
    c8 = fits.Column(name='PETAL', format='12A', array=meta_petal[i*chunk_size:(i+1)*chunk_size])
    c9 = fits.Column(name='TILE', format='12A', array=meta_tile[i*chunk_size:(i+1)*chunk_size])
    hdu_meta = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9], name='METADATA')
    hdu_meta.header['BLINDING'] = 'none'

    hdu_delta = fits.ImageHDU(delta[spender_mask], name=f'DELTA')
    hdu_weight = fits.ImageHDU(weight_chunks[ichunk], name=f'WEIGHT')
    hdu_cont = fits.ImageHDU(_cont_chunks[ichunk], name=f'CONT')
    hdu_tff_bar = fits.ImageHDU(tff_bar, name=f'FBAR')

    # Combine all HDUs into an HDUList
    hdul = fits.HDUList([primary_hdu, hdu_wave, hdu_meta, hdu_delta, hdu_weight, hdu_cont, hdu_tff_bar])

    # Write the HDUList to a new FITS file
    hdul.writeto(f'{outpath}/delta-%i.fits' % ichunk, overwrite=True)
    i+=1
        

# code with bug

In [11]:
def chunks(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]


In [12]:
def spender_to_deltas(ibatchrange = 50):

    #loop over ibatch values in "grab values from spenderq files"
    #ibatch values in groups of 50, so if ibatchrange is 100, the range is (50,100)
    # for ibatch in range(0,2):
    if len(ibatchrange) > 1:
        start = ibatchrange[0]
        end = ibatchrange[1]
    else:
        start = ibatchrange-50
        end = ibatchrange
    
    for ibatch in range(start, end):
        normalised_flux, pipeline_weight, z, tid, normalisation, zerr = [], [], [], [], [], []

        sq_reconstructions = []
        print('ibatch', ibatch)

        #load batch
        with open(f'{path}/{file_prefix}_%i.pkl' % ibatch, 'rb') as f:
            _normalised_flux, _pipeline_weight, _z, _tid, _normalisation, _zerr = pickle.load(f)
        normalised_flux.append(np.array(_normalised_flux))
        pipeline_weight.append(np.array(_pipeline_weight))
        z.append(np.array(_z))
        tid.append(np.array(_tid))
        normalisation.append(np.array(_normalisation))
        zerr.append(np.array(_zerr))
        
        #load SpenderQ reconstructions
        _sq_reconstructions = np.load(f'{path}/{file_prefix}_%i.recons.npy' % ibatch)
        sq_reconstructions.append(np.array(_sq_reconstructions))

        normalised_flux=np.concatenate(normalised_flux,axis=0)
        pipeline_weight=np.concatenate(pipeline_weight,axis=0)
        z=np.concatenate(z)
        tid=np.concatenate(tid)
        normalisation=np.concatenate(normalisation,axis=0)
        zerr=np.concatenate(zerr,axis=0)
        sq_reconstructions=np.concatenate(sq_reconstructions,axis=0)

        #create variables needed for the delta files
        #nb_of_quasars = 10
        nb_of_quasars = len(z)
        
        _tff = np.full((nb_of_quasars,7781),np.nan)
        _weight = np.full((nb_of_quasars,7781),np.nan)
        _sq_cont = np.full((nb_of_quasars,7781),np.nan)
        
        wrecon = np.load(f'{path}/{file_prefix}.wave_recon.npy')
        desi_wave = np.linspace(3600, 9824, 7781) #obs
        
        _tff_bar_up = np.zeros(7781)
        _tff_bar_down = np.zeros(7781)
        
        #for iqso in range(10):
        for iqso in range(nb_of_quasars):
        
            #create wavelength grids
            desi_wave_rest = desi_wave/(1+z[iqso]) #rest
            
            #rebin spender reconstruction
            edges = np.linspace(wrecon[0], wrecon[-1], 7782)
            spenderq_rebin = U.trapz_rebin(wrecon, sq_reconstructions[iqso], edges = edges)
            
            #keep only part of spec that is within the lya range
            mask_desi_rest = (desi_wave_rest >= forest_lambda_min) & (desi_wave_rest <= forest_lambda_max)
        
            _tff[iqso][mask_desi_rest] = normalised_flux[iqso][mask_desi_rest]/spenderq_rebin[mask_desi_rest]
            _weight[iqso][mask_desi_rest] = pipeline_weight[iqso][mask_desi_rest]
            _sq_cont[iqso][mask_desi_rest] = spenderq_rebin[mask_desi_rest]
            _tff_bar_up += normalised_flux[iqso]/spenderq_rebin * pipeline_weight[iqso]
            _tff_bar_down += pipeline_weight[iqso]

        #get the weighted average transmitted flux fraction 
        picca_range = (desi_wave <= LAMBDA[-1])
        tff_bar = np.divide(_tff_bar_up[picca_range], _tff_bar_down[picca_range], out=np.zeros_like(_tff_bar_up[picca_range]), where=_tff_bar_down[picca_range]!=0)

        #picca structure: nan filled arrays
        delta = np.full((nb_of_quasars,sum(picca_range)),np.nan)
        weight = np.full((nb_of_quasars,sum(picca_range)),np.nan)
        sq_cont = np.full((nb_of_quasars,sum(picca_range)),np.nan)
        
        meta_los_id, meta_ra, meta_dec, meta_z, meta_meansnr, meta_targetid, meta_night, meta_petal, meta_tile = [], [], [], [], [], [], [], [], []
        
        #for iqso in range(5):
        for iqso in range(nb_of_quasars):
            
            delta[iqso] = (_tff[iqso][picca_range]/tff_bar) - 1.
            weight[iqso] = _weight[iqso][picca_range]
            sq_cont[iqso] = _sq_cont[iqso][picca_range]
            
            los_id_idx = int(np.where(catalogue_TID == int(tid[iqso]))[0][0])
            
            meta_los_id.append(int(catalogue_TID[los_id_idx]))
            meta_ra.append(float(math.radians(catalogue_RA[los_id_idx])))
            meta_dec.append(float(math.radians(catalogue_DEC[los_id_idx])))
            meta_z.append(float(catalogue_Z[los_id_idx]))
            meta_meansnr.append(np.nan)
            meta_targetid.append(int(catalogue_TID[los_id_idx]))
            meta_night.append('')
            meta_petal.append('')
            meta_tile.append('')

        # Create primary HDU 
        primary_hdu = fits.PrimaryHDU(data=None)
        
        # Create OBSERVED WAVELENGTH HDU
        hdu_wave = fits.ImageHDU(LAMBDA, name=f'LAMBDA')
        hdu_wave.header['HIERARCH WAVE_SOLUTION'] = 'lin'
        hdu_wave.header['HIERARCH DELTA_LAMBDA'] = 0.8
        
        # Set chunk size, and how to chunk
        chunk_size = 1024
        
        
        
        _delta_chunks = chunks(delta, chunk_size)
        _weight_chunks = chunks(weight, chunk_size)
        _cont_chunks = chunks(sq_cont, chunk_size)
        
        nb_of_chunks = len(_delta_chunks)
        print('num chunks', nb_of_chunks)
        
        for ichunk in range(nb_of_chunks):
            i=0
            c1 = fits.Column(name='LOS_ID', format='K', array=np.array(meta_los_id[i*chunk_size:(i+1)*chunk_size]))
            c2 = fits.Column(name='RA', format='D', array=np.array(meta_ra[i*chunk_size:(i+1)*chunk_size]))
            c3 = fits.Column(name='DEC', format='D', array=np.array(meta_dec[i*chunk_size:(i+1)*chunk_size]))
            c4 = fits.Column(name='Z', format='D', array=np.array(meta_z[i*chunk_size:(i+1)*chunk_size]))
            c5 = fits.Column(name='MEANSNR', format='D', array=meta_meansnr[i*chunk_size:(i+1)*chunk_size])
            c6 = fits.Column(name='TARGETID', format='K', array=np.array(meta_targetid[i*chunk_size:(i+1)*chunk_size]))
            c7 = fits.Column(name='NIGHT', format='12A', array=meta_night[i*chunk_size:(i+1)*chunk_size])
            c8 = fits.Column(name='PETAL', format='12A', array=meta_petal[i*chunk_size:(i+1)*chunk_size])
            c9 = fits.Column(name='TILE', format='12A', array=meta_tile[i*chunk_size:(i+1)*chunk_size])
            hdu_meta = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9], name='METADATA')
            hdu_meta.header['BLINDING'] = 'none'
        
            hdu_delta = fits.ImageHDU(_delta_chunks[ichunk], name=f'DELTA')
            hdu_weight = fits.ImageHDU(_weight_chunks[ichunk], name=f'WEIGHT')
            hdu_cont = fits.ImageHDU(_cont_chunks[ichunk], name=f'CONT')
            hdu_tff_bar = fits.ImageHDU(tff_bar, name=f'FBAR')
        
            # Combine all HDUs into an HDUList
            hdul = fits.HDUList([primary_hdu, hdu_wave, hdu_meta, hdu_delta, hdu_weight, hdu_cont, hdu_tff_bar])
        
            # Write the HDUList to a new FITS file
            hdul.writeto(f'{outpath}/delta-%i.fits' % ibatch, overwrite=True)
            i+=1
                

In [20]:
spender_to_deltas()

ibatch 0
num chunks 1
ibatch 1
num chunks 1
ibatch 2
num chunks 1
ibatch 3
num chunks 1
ibatch 4
num chunks 1
ibatch 5
num chunks 1
ibatch 6
num chunks 1
ibatch 7
num chunks 1
ibatch 8
num chunks 1
ibatch 9
num chunks 1
ibatch 10
num chunks 1
ibatch 11
num chunks 1
ibatch 12
num chunks 1
ibatch 13
num chunks 1
ibatch 14
num chunks 1
ibatch 15
num chunks 1
ibatch 16
num chunks 1
ibatch 17
num chunks 1
ibatch 18
num chunks 1
ibatch 19
num chunks 1
ibatch 20
num chunks 1
ibatch 21
num chunks 1
ibatch 22
num chunks 1
ibatch 23
num chunks 1
ibatch 24
num chunks 1
ibatch 25
num chunks 1
ibatch 26
num chunks 1
ibatch 27
num chunks 1
ibatch 28
num chunks 1
ibatch 29
num chunks 1
ibatch 30
num chunks 1
ibatch 31
num chunks 1
ibatch 32
num chunks 1
ibatch 33
num chunks 1
ibatch 34
num chunks 1
ibatch 35
num chunks 1
ibatch 36
num chunks 1
ibatch 37
num chunks 1
ibatch 38
num chunks 1
ibatch 39
num chunks 1
ibatch 40
num chunks 1
ibatch 41
num chunks 1
ibatch 42
num chunks 1
ibatch 43
num chunks 

In [22]:
spender_to_deltas(ibatchrange = 100)

ibatch 50
num chunks 1
ibatch 51
num chunks 1
ibatch 52
num chunks 1
ibatch 53
num chunks 1
ibatch 54
num chunks 1
ibatch 55
num chunks 1
ibatch 56
num chunks 1
ibatch 57
num chunks 1
ibatch 58
num chunks 1
ibatch 59
num chunks 1
ibatch 60
num chunks 1
ibatch 61
num chunks 1
ibatch 62
num chunks 1
ibatch 63
num chunks 1
ibatch 64
num chunks 1
ibatch 65
num chunks 1
ibatch 66
num chunks 1
ibatch 67
num chunks 1
ibatch 68
num chunks 1
ibatch 69
num chunks 1
ibatch 70
num chunks 1
ibatch 71
num chunks 1
ibatch 72
num chunks 1
ibatch 73
num chunks 1
ibatch 74
num chunks 1
ibatch 75
num chunks 1
ibatch 76
num chunks 1
ibatch 77
num chunks 1
ibatch 78
num chunks 1
ibatch 79
num chunks 1
ibatch 80
num chunks 1
ibatch 81
num chunks 1
ibatch 82
num chunks 1
ibatch 83
num chunks 1
ibatch 84
num chunks 1
ibatch 85
num chunks 1
ibatch 86
num chunks 1
ibatch 87
num chunks 1
ibatch 88
num chunks 1
ibatch 89
num chunks 1
ibatch 90
num chunks 1
ibatch 91
num chunks 1
ibatch 92
num chunks 1
ibatch 93
n

In [13]:
spender_to_deltas(ibatchrange = (100, 393))

ibatch 100
num chunks 1
ibatch 101
num chunks 1
ibatch 102
num chunks 1
ibatch 103
num chunks 1
ibatch 104
num chunks 1
ibatch 105
num chunks 1
ibatch 106
num chunks 1
ibatch 107
num chunks 1
ibatch 108
num chunks 1
ibatch 109
num chunks 1
ibatch 110
num chunks 1
ibatch 111
num chunks 1
ibatch 112
num chunks 1
ibatch 113
num chunks 1
ibatch 114
num chunks 1
ibatch 115
num chunks 1
ibatch 116
num chunks 1
ibatch 117
num chunks 1
ibatch 118
num chunks 1
ibatch 119
num chunks 1
ibatch 120
num chunks 1
ibatch 121
num chunks 1
ibatch 122
num chunks 1
ibatch 123
num chunks 1
ibatch 124
num chunks 1
ibatch 125
num chunks 1
ibatch 126
num chunks 1
ibatch 127
num chunks 1
ibatch 128
num chunks 1
ibatch 129
num chunks 1
ibatch 130
num chunks 1
ibatch 131
num chunks 1
ibatch 132
num chunks 1
ibatch 133
num chunks 1
ibatch 134
num chunks 1
ibatch 135
num chunks 1
ibatch 136
num chunks 1
ibatch 137
num chunks 1
ibatch 138
num chunks 1
ibatch 139
num chunks 1
ibatch 140
num chunks 1
ibatch 141
num c

# old code that was copied into the above function

## Grab values from SpenderQ files

In [11]:
normalised_flux, pipeline_weight, z, tid, normalisation, zerr = [], [], [], [], [], []

sq_reconstructions = []

for ibatch in range(0,1):
    print(ibatch)
#for ibatch in range(len(spender_output_files)): 
    
    #load batch
    with open(f'{path}/{file_prefix}_%i.pkl' % ibatch, 'rb') as f:
        _normalised_flux, _pipeline_weight, _z, _tid, _normalisation, _zerr = pickle.load(f)
    normalised_flux.append(np.array(_normalised_flux))
    pipeline_weight.append(np.array(_pipeline_weight))
    z.append(np.array(_z))
    tid.append(np.array(_tid))
    normalisation.append(np.array(_normalisation))
    zerr.append(np.array(_zerr))
    
    #load SpenderQ reconstructions
    _sq_reconstructions = np.load(f'{path}/{file_prefix}_%i.recons.npy' % ibatch)
    sq_reconstructions.append(np.array(_sq_reconstructions))

normalised_flux=np.concatenate(normalised_flux,axis=0)
pipeline_weight=np.concatenate(pipeline_weight,axis=0)
z=np.concatenate(z)
tid=np.concatenate(tid)
normalisation=np.concatenate(normalisation,axis=0)
zerr=np.concatenate(zerr,axis=0)
sq_reconstructions=np.concatenate(sq_reconstructions,axis=0)

0


## Create variables needed for delta files

In [12]:
#nb_of_quasars = 10
nb_of_quasars = len(z)

_tff = np.full((nb_of_quasars,7781),np.nan)
_weight = np.full((nb_of_quasars,7781),np.nan)
_sq_cont = np.full((nb_of_quasars,7781),np.nan)

wrecon = np.load(f'{path}/{file_prefix}.wave_recon.npy')
desi_wave = np.linspace(3600, 9824, 7781) #obs

_tff_bar_up = np.zeros(7781)
_tff_bar_down = np.zeros(7781)

#for iqso in range(10):
for iqso in range(nb_of_quasars):

    #create wavelength grids
    desi_wave_rest = desi_wave/(1+z[iqso]) #rest
    
    #rebin spender reconstruction
    edges = np.linspace(wrecon[0], wrecon[-1], 7782)
    spenderq_rebin = U.trapz_rebin(wrecon, sq_reconstructions[iqso], edges = edges)
    
    #keep only part of spec that is within the lya range
    mask_desi_rest = (desi_wave_rest >= forest_lambda_min) & (desi_wave_rest <= forest_lambda_max)

    _tff[iqso][mask_desi_rest] = normalised_flux[iqso][mask_desi_rest]/spenderq_rebin[mask_desi_rest]
    _weight[iqso][mask_desi_rest] = pipeline_weight[iqso][mask_desi_rest]
    _sq_cont[iqso][mask_desi_rest] = spenderq_rebin[mask_desi_rest]
    _tff_bar_up += normalised_flux[iqso]/spenderq_rebin * pipeline_weight[iqso]
    _tff_bar_down += pipeline_weight[iqso]


In [13]:
#get the weighted average transmitted flux fraction 
picca_range = (desi_wave <= LAMBDA[-1])
tff_bar = np.divide(_tff_bar_up[picca_range], _tff_bar_down[picca_range], out=np.zeros_like(_tff_bar_up[picca_range]), where=_tff_bar_down[picca_range]!=0)

In [14]:
#picca structure: nan filled arrays
delta = np.full((nb_of_quasars,sum(picca_range)),np.nan)
weight = np.full((nb_of_quasars,sum(picca_range)),np.nan)
sq_cont = np.full((nb_of_quasars,sum(picca_range)),np.nan)

meta_los_id, meta_ra, meta_dec, meta_z, meta_meansnr, meta_targetid, meta_night, meta_petal, meta_tile = [], [], [], [], [], [], [], [], []

#for iqso in range(5):
for iqso in range(nb_of_quasars):
    
    delta[iqso] = (_tff[iqso][picca_range]/tff_bar) - 1.
    weight[iqso] = _weight[iqso][picca_range]
    sq_cont[iqso] = _sq_cont[iqso][picca_range]
    
    los_id_idx = int(np.where(catalogue_TID == int(tid[iqso]))[0][0])
    
    meta_los_id.append(int(catalogue_TID[los_id_idx]))
    meta_ra.append(float(math.radians(catalogue_RA[los_id_idx])))
    meta_dec.append(float(math.radians(catalogue_DEC[los_id_idx])))
    meta_z.append(float(catalogue_Z[los_id_idx]))
    meta_meansnr.append(np.nan)
    meta_targetid.append(int(catalogue_TID[los_id_idx]))
    meta_night.append('')
    meta_petal.append('')
    meta_tile.append('')
    

## Create the delta files

In [15]:
# Create primary HDU 
primary_hdu = fits.PrimaryHDU(data=None)

# Create OBSERVED WAVELENGTH HDU
hdu_wave = fits.ImageHDU(LAMBDA, name=f'LAMBDA')
hdu_wave.header['HIERARCH WAVE_SOLUTION'] = 'lin'
hdu_wave.header['HIERARCH DELTA_LAMBDA'] = 0.8

# Set chunk size, and how to chunk
chunk_size = 1024

def chunks(lst, n):
    return [lst[i:i + n] for i in range(0, len(lst), n)]

_delta_chunks = chunks(delta, chunk_size)
_weight_chunks = chunks(weight, chunk_size)
_cont_chunks = chunks(sq_cont, chunk_size)

nb_of_chunks = len(_delta_chunks)
print(nb_of_chunks)

for ichunk in range(nb_of_chunks):
    i=0
    c1 = fits.Column(name='LOS_ID', format='K', array=np.array(meta_los_id[i*chunk_size:(i+1)*chunk_size]))
    c2 = fits.Column(name='RA', format='D', array=np.array(meta_ra[i*chunk_size:(i+1)*chunk_size]))
    c3 = fits.Column(name='DEC', format='D', array=np.array(meta_dec[i*chunk_size:(i+1)*chunk_size]))
    c4 = fits.Column(name='Z', format='D', array=np.array(meta_z[i*chunk_size:(i+1)*chunk_size]))
    c5 = fits.Column(name='MEANSNR', format='D', array=meta_meansnr[i*chunk_size:(i+1)*chunk_size])
    c6 = fits.Column(name='TARGETID', format='K', array=np.array(meta_targetid[i*chunk_size:(i+1)*chunk_size]))
    c7 = fits.Column(name='NIGHT', format='12A', array=meta_night[i*chunk_size:(i+1)*chunk_size])
    c8 = fits.Column(name='PETAL', format='12A', array=meta_petal[i*chunk_size:(i+1)*chunk_size])
    c9 = fits.Column(name='TILE', format='12A', array=meta_tile[i*chunk_size:(i+1)*chunk_size])
    hdu_meta = fits.BinTableHDU.from_columns([c1, c2, c3, c4, c5, c6, c7, c8, c9], name='METADATA')
    hdu_meta.header['BLINDING'] = 'none'

    hdu_delta = fits.ImageHDU(_delta_chunks[ichunk], name=f'DELTA')
    hdu_weight = fits.ImageHDU(_weight_chunks[ichunk], name=f'WEIGHT')
    hdu_cont = fits.ImageHDU(_cont_chunks[ichunk], name=f'CONT')
    hdu_tff_bar = fits.ImageHDU(tff_bar, name=f'FBAR')

    # Combine all HDUs into an HDUList
    hdul = fits.HDUList([primary_hdu, hdu_wave, hdu_meta, hdu_delta, hdu_weight, hdu_cont, hdu_tff_bar])

    # Write the HDUList to a new FITS file
    hdul.writeto(f'{outpath}/delta-%i.fits' % ichunk, overwrite=True)
    i+=1
        

1
